In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import Adam
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [2]:
#get displacement for one dot over the whole time 60 20 20
#open file by file
CSVpath="InSAR_data_south/displacement/CSV/test/"
dir_list=[]
lon_south=pd.read_csv('InSAR_data_south/longitude.csv')
lat_south=pd.read_csv('InSAR_data_south/latitude.csv')
for f in os.listdir(CSVpath):
    name, ext=os.path.splitext(f)
    if(ext=='.csv'):
        dir_list.append(f)

dir_list.sort()
print(dir_list)


['20141108.csv', '20141202.csv', '20141226.csv', '20150212.csv', '20150308.csv', '20150401.csv', '20150425.csv', '20150519.csv', '20150612.csv', '20150706.csv', '20150730.csv', '20150823.csv', '20150916.csv']


In [3]:
# df=pd.DataFrame()
# for f in dir_list:
#     name, ext=os.path.splitext(f)
#     if(ext=='.csv'):
#         print(CSVpath+f)
#         df_temp=pd.read_csv(CSVpath+f)
#         df_temp.columns=lon_south.columns
#         df_temp.index=lat_south.columns[:-1]
#         df_temp=df_temp.unstack().reset_index()
#         df_temp.columns=['Longitude','Latitude',name]
#         #print(df1.head())
#         #put it onto the main'
#         df1=pd.DataFrame(data=df_temp.iloc[:,2:3])
#         df1=df_temp.set_index([df_temp.columns[0],df_temp.columns[1]])
        
        
#         df=pd.concat([df,df1], axis=1)
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')




In [4]:
# Convert dates to year-months
#df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
dftest=df
timestep=10
#dftest


In [5]:
#df

In [6]:
#df.query("Longitude==-117.6414 and Latitude==37.6236")
# print(dftest.query('20141108'))
#df   

In [7]:
#tup=(-119.9588,35.185)
tup=(-119.2990,35.8852)
#tup=(-120.2888,34.8349)
#tup=(-117.6414,37.6236)
#print(dftest.loc[tup])

In [8]:
def shaping(datain, timestep):
    
    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten() 
    arr[np.isnan(arr)] = -100
    #print(arr)
    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.  
        
        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr[X_start:X_end]
            Y_comb=arr[Y_start:Y_end]
        else: 
            X_comb=np.append(X_comb, arr[X_start:X_end])
            Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [9]:
##### Step 2 - Prepare data
tup=(-119.2990,35.8852)
# Split data into train and test dataframes
df_train_temp=dftest.iloc[:, :70].copy()
df_test_temp=dftest.iloc[:, 70:110].copy()
#pick a location to test
df_train = df_train_temp[df_train_temp.index==tup].copy()
df_test = df_test_temp[df_test_temp.index==tup].copy()

print(df_test.columns)
#print(df_test)
#df_train.columns=['Date','Disp']
#df_test.columns=['Date','Disp']


# # Split data into train and test dataframes
# df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
# df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# # Select one location
# dfloc_train = df_train[df_train.index==location].copy()
# dfloc_test = df_test[df_test.index==location].copy()
# #print(df_test.columns)
# Select one location
#dfloc_train = df_train[df_train.index==location].copy()
#dfloc_test = df_test[df_test.index==location].copy()

# Use previously defined shaping function to reshape the data for LSTM

X_train, Y_train = shaping(datain=df_train, timestep= timestep)
X_test, Y_test = shaping(datain=df_test, timestep=timestep)

DatetimeIndex(['2017-10-23', '2017-11-04', '2017-11-16', '2017-11-28',
               '2017-12-10', '2017-12-22', '2018-01-03', '2018-01-15',
               '2018-01-27', '2018-02-08', '2018-02-20', '2018-03-04',
               '2018-03-16', '2018-03-28', '2018-04-09', '2018-04-21',
               '2018-05-03', '2018-05-15', '2018-05-27', '2018-06-08',
               '2018-06-20', '2018-07-02', '2018-07-14', '2018-07-26',
               '2018-08-07', '2018-08-19', '2018-08-31', '2018-09-12',
               '2018-09-24', '2018-10-06', '2018-10-18', '2018-10-30',
               '2018-11-11', '2018-11-17', '2018-11-23', '2018-12-05',
               '2018-12-17', '2018-12-29', '2019-01-10', '2019-01-22'],
              dtype='datetime64[ns]', freq=None)


In [10]:
#print("dftrain:", df_train)

#print("dftest:", df_test)

#print("Xtrain:", X_train)

#print("Ytrain:", Y_train.shape[2])

#print("Xtrain:", X_train.size)

In [11]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Masking(mask_value=-100, input_shape=(timestep, 1)))
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x



In [12]:
##### Step 4 - Compile the model
#from keras.optimizers import adam_v2
#optimizer = adam_v2.Adam(learning_rate=1)
optimizer=Adam( 0.001,0.0001)

model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
        
             )

In [26]:


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=None, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=5000, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=2, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                
                   )



Epoch 1/5000
2/2 - 0s - loss: 0.3154 - mean_squared_error: 0.3154 - mean_absolute_error: 0.3931 - 26ms/epoch - 13ms/step
Epoch 2/5000
2/2 - 0s - loss: 0.3141 - mean_squared_error: 0.3141 - mean_absolute_error: 0.3906 - 23ms/epoch - 11ms/step
Epoch 3/5000
2/2 - 0s - loss: 0.3076 - mean_squared_error: 0.3076 - mean_absolute_error: 0.3872 - 23ms/epoch - 11ms/step
Epoch 4/5000
2/2 - 0s - loss: 0.3113 - mean_squared_error: 0.3113 - mean_absolute_error: 0.3927 - 23ms/epoch - 11ms/step
Epoch 5/5000
2/2 - 0s - loss: 0.3065 - mean_squared_error: 0.3065 - mean_absolute_error: 0.3868 - 22ms/epoch - 11ms/step
Epoch 6/5000
2/2 - 0s - loss: 0.3096 - mean_squared_error: 0.3096 - mean_absolute_error: 0.3894 - 23ms/epoch - 11ms/step
Epoch 7/5000
2/2 - 0s - loss: 0.3131 - mean_squared_error: 0.3131 - mean_absolute_error: 0.3957 - 25ms/epoch - 12ms/step
Epoch 8/5000
2/2 - 0s - loss: 0.3082 - mean_squared_error: 0.3082 - mean_absolute_error: 0.3895 - 22ms/epoch - 11ms/step
Epoch 9/5000
2/2 - 0s - loss: 0.

Epoch 69/5000
2/2 - 0s - loss: 0.2862 - mean_squared_error: 0.2862 - mean_absolute_error: 0.3605 - 22ms/epoch - 11ms/step
Epoch 70/5000
2/2 - 0s - loss: 0.2877 - mean_squared_error: 0.2877 - mean_absolute_error: 0.3617 - 21ms/epoch - 10ms/step
Epoch 71/5000
2/2 - 0s - loss: 0.2836 - mean_squared_error: 0.2836 - mean_absolute_error: 0.3578 - 21ms/epoch - 10ms/step
Epoch 72/5000
2/2 - 0s - loss: 0.2837 - mean_squared_error: 0.2837 - mean_absolute_error: 0.3575 - 21ms/epoch - 10ms/step
Epoch 73/5000
2/2 - 0s - loss: 0.2856 - mean_squared_error: 0.2856 - mean_absolute_error: 0.3586 - 22ms/epoch - 11ms/step
Epoch 74/5000
2/2 - 0s - loss: 0.2826 - mean_squared_error: 0.2826 - mean_absolute_error: 0.3570 - 23ms/epoch - 11ms/step
Epoch 75/5000
2/2 - 0s - loss: 0.2839 - mean_squared_error: 0.2839 - mean_absolute_error: 0.3571 - 24ms/epoch - 12ms/step
Epoch 76/5000
2/2 - 0s - loss: 0.2814 - mean_squared_error: 0.2814 - mean_absolute_error: 0.3537 - 25ms/epoch - 12ms/step
Epoch 77/5000
2/2 - 0s -

Epoch 136/5000
2/2 - 0s - loss: 0.2704 - mean_squared_error: 0.2704 - mean_absolute_error: 0.3395 - 22ms/epoch - 11ms/step
Epoch 137/5000
2/2 - 0s - loss: 0.2744 - mean_squared_error: 0.2744 - mean_absolute_error: 0.3425 - 21ms/epoch - 10ms/step
Epoch 138/5000
2/2 - 0s - loss: 0.2728 - mean_squared_error: 0.2728 - mean_absolute_error: 0.3441 - 21ms/epoch - 10ms/step
Epoch 139/5000
2/2 - 0s - loss: 0.2814 - mean_squared_error: 0.2814 - mean_absolute_error: 0.3593 - 21ms/epoch - 10ms/step
Epoch 140/5000
2/2 - 0s - loss: 0.3033 - mean_squared_error: 0.3033 - mean_absolute_error: 0.3872 - 21ms/epoch - 10ms/step
Epoch 141/5000
2/2 - 0s - loss: 0.2831 - mean_squared_error: 0.2831 - mean_absolute_error: 0.3606 - 22ms/epoch - 11ms/step
Epoch 142/5000
2/2 - 0s - loss: 0.2881 - mean_squared_error: 0.2881 - mean_absolute_error: 0.3660 - 21ms/epoch - 10ms/step
Epoch 143/5000
2/2 - 0s - loss: 0.2858 - mean_squared_error: 0.2858 - mean_absolute_error: 0.3689 - 22ms/epoch - 11ms/step
Epoch 144/5000
2

Epoch 202/5000
2/2 - 0s - loss: 0.2667 - mean_squared_error: 0.2667 - mean_absolute_error: 0.3369 - 24ms/epoch - 12ms/step
Epoch 203/5000
2/2 - 0s - loss: 0.2638 - mean_squared_error: 0.2638 - mean_absolute_error: 0.3348 - 22ms/epoch - 11ms/step
Epoch 204/5000
2/2 - 0s - loss: 0.2634 - mean_squared_error: 0.2634 - mean_absolute_error: 0.3309 - 22ms/epoch - 11ms/step
Epoch 205/5000
2/2 - 0s - loss: 0.2629 - mean_squared_error: 0.2629 - mean_absolute_error: 0.3312 - 22ms/epoch - 11ms/step
Epoch 206/5000
2/2 - 0s - loss: 0.2651 - mean_squared_error: 0.2651 - mean_absolute_error: 0.3347 - 23ms/epoch - 11ms/step
Epoch 207/5000
2/2 - 0s - loss: 0.2612 - mean_squared_error: 0.2612 - mean_absolute_error: 0.3317 - 23ms/epoch - 11ms/step
Epoch 208/5000
2/2 - 0s - loss: 0.2630 - mean_squared_error: 0.2630 - mean_absolute_error: 0.3355 - 33ms/epoch - 16ms/step
Epoch 209/5000
2/2 - 0s - loss: 0.2764 - mean_squared_error: 0.2764 - mean_absolute_error: 0.3525 - 25ms/epoch - 13ms/step
Epoch 210/5000
2

Epoch 269/5000
2/2 - 0s - loss: 0.2686 - mean_squared_error: 0.2686 - mean_absolute_error: 0.3528 - 24ms/epoch - 12ms/step
Epoch 270/5000
2/2 - 0s - loss: 0.2669 - mean_squared_error: 0.2669 - mean_absolute_error: 0.3431 - 24ms/epoch - 12ms/step
Epoch 271/5000
2/2 - 0s - loss: 0.2719 - mean_squared_error: 0.2719 - mean_absolute_error: 0.3522 - 23ms/epoch - 11ms/step
Epoch 272/5000
2/2 - 0s - loss: 0.2638 - mean_squared_error: 0.2638 - mean_absolute_error: 0.3391 - 24ms/epoch - 12ms/step
Epoch 273/5000
2/2 - 0s - loss: 0.2639 - mean_squared_error: 0.2639 - mean_absolute_error: 0.3418 - 23ms/epoch - 11ms/step
Epoch 274/5000
2/2 - 0s - loss: 0.2717 - mean_squared_error: 0.2717 - mean_absolute_error: 0.3572 - 21ms/epoch - 10ms/step
Epoch 275/5000
2/2 - 0s - loss: 0.2652 - mean_squared_error: 0.2652 - mean_absolute_error: 0.3438 - 22ms/epoch - 11ms/step
Epoch 276/5000
2/2 - 0s - loss: 0.2612 - mean_squared_error: 0.2612 - mean_absolute_error: 0.3391 - 22ms/epoch - 11ms/step
Epoch 277/5000
2

...

2/2 - 0s - loss: 0.1597 - mean_squared_error: 0.1597 - mean_absolute_error: 0.1967 - 22ms/epoch - 11ms/step
Epoch 4698/5000
2/2 - 0s - loss: 0.1477 - mean_squared_error: 0.1477 - mean_absolute_error: 0.1862 - 23ms/epoch - 11ms/step
Epoch 4699/5000
2/2 - 0s - loss: 0.1497 - mean_squared_error: 0.1497 - mean_absolute_error: 0.1848 - 21ms/epoch - 10ms/step
Epoch 4700/5000
2/2 - 0s - loss: 0.1611 - mean_squared_error: 0.1611 - mean_absolute_error: 0.2027 - val_loss: 27.7574 - val_mean_squared_error: 27.7574 - val_mean_absolute_error: 4.3402 - 46ms/epoch - 23ms/step
Epoch 4701/5000
2/2 - 0s - loss: 0.1482 - mean_squared_error: 0.1482 - mean_absolute_error: 0.1912 - 22ms/epoch - 11ms/step
Epoch 4702/5000
2/2 - 0s - loss: 0.1472 - mean_squared_error: 0.1472 - mean_absolute_error: 0.1848 - 22ms/epoch - 11ms/step
Epoch 4703/5000
2/2 - 0s - loss: 0.1540 - mean_squared_error: 0.1540 - mean_absolute_error: 0.2035 - 23ms/epoch - 11ms/step
Epoch 4704/5000
2/2 - 0s - loss: 0.1559 - mean_squared_error

Epoch 4763/5000
2/2 - 0s - loss: 0.1502 - mean_squared_error: 0.1502 - mean_absolute_error: 0.1855 - 22ms/epoch - 11ms/step
Epoch 4764/5000
2/2 - 0s - loss: 0.1473 - mean_squared_error: 0.1473 - mean_absolute_error: 0.1798 - 22ms/epoch - 11ms/step
Epoch 4765/5000
2/2 - 0s - loss: 0.1540 - mean_squared_error: 0.1540 - mean_absolute_error: 0.1881 - 22ms/epoch - 11ms/step
Epoch 4766/5000
2/2 - 0s - loss: 0.1590 - mean_squared_error: 0.1590 - mean_absolute_error: 0.1933 - 23ms/epoch - 11ms/step
Epoch 4767/5000
2/2 - 0s - loss: 0.1521 - mean_squared_error: 0.1521 - mean_absolute_error: 0.1933 - 22ms/epoch - 11ms/step
Epoch 4768/5000
2/2 - 0s - loss: 0.1492 - mean_squared_error: 0.1492 - mean_absolute_error: 0.1855 - 22ms/epoch - 11ms/step
Epoch 4769/5000
2/2 - 0s - loss: 0.1638 - mean_squared_error: 0.1638 - mean_absolute_error: 0.2116 - 22ms/epoch - 11ms/step
Epoch 4770/5000
2/2 - 0s - loss: 0.1543 - mean_squared_error: 0.1543 - mean_absolute_error: 0.1930 - 22ms/epoch - 11ms/step
Epoch 47

Epoch 4829/5000
2/2 - 0s - loss: 0.1475 - mean_squared_error: 0.1475 - mean_absolute_error: 0.1821 - 22ms/epoch - 11ms/step
Epoch 4830/5000
2/2 - 0s - loss: 0.1518 - mean_squared_error: 0.1518 - mean_absolute_error: 0.1929 - 22ms/epoch - 11ms/step
Epoch 4831/5000
2/2 - 0s - loss: 0.1498 - mean_squared_error: 0.1498 - mean_absolute_error: 0.2001 - 21ms/epoch - 10ms/step
Epoch 4832/5000
2/2 - 0s - loss: 0.1459 - mean_squared_error: 0.1459 - mean_absolute_error: 0.1847 - 22ms/epoch - 11ms/step
Epoch 4833/5000
2/2 - 0s - loss: 0.1516 - mean_squared_error: 0.1516 - mean_absolute_error: 0.1892 - 21ms/epoch - 10ms/step
Epoch 4834/5000
2/2 - 0s - loss: 0.1760 - mean_squared_error: 0.1760 - mean_absolute_error: 0.2082 - 22ms/epoch - 11ms/step
Epoch 4835/5000
2/2 - 0s - loss: 0.1834 - mean_squared_error: 0.1834 - mean_absolute_error: 0.2177 - 22ms/epoch - 11ms/step
Epoch 4836/5000
2/2 - 0s - loss: 0.1853 - mean_squared_error: 0.1853 - mean_absolute_error: 0.2261 - 21ms/epoch - 10ms/step
Epoch 48

2/2 - 0s - loss: 0.1672 - mean_squared_error: 0.1672 - mean_absolute_error: 0.2243 - 21ms/epoch - 10ms/step
Epoch 4896/5000
2/2 - 0s - loss: 0.1707 - mean_squared_error: 0.1707 - mean_absolute_error: 0.2193 - 22ms/epoch - 11ms/step
Epoch 4897/5000
2/2 - 0s - loss: 0.1477 - mean_squared_error: 0.1477 - mean_absolute_error: 0.1868 - 21ms/epoch - 10ms/step
Epoch 4898/5000
2/2 - 0s - loss: 0.1445 - mean_squared_error: 0.1445 - mean_absolute_error: 0.1787 - 22ms/epoch - 11ms/step
Epoch 4899/5000
2/2 - 0s - loss: 0.1478 - mean_squared_error: 0.1478 - mean_absolute_error: 0.1782 - 22ms/epoch - 11ms/step
Epoch 4900/5000
2/2 - 0s - loss: 0.1481 - mean_squared_error: 0.1481 - mean_absolute_error: 0.1793 - val_loss: 27.2358 - val_mean_squared_error: 27.2358 - val_mean_absolute_error: 4.3013 - 46ms/epoch - 23ms/step
Epoch 4901/5000
2/2 - 0s - loss: 0.1483 - mean_squared_error: 0.1483 - mean_absolute_error: 0.1840 - 22ms/epoch - 11ms/step
Epoch 4902/5000
2/2 - 0s - loss: 0.1485 - mean_squared_error

Epoch 4961/5000
2/2 - 0s - loss: 0.1643 - mean_squared_error: 0.1643 - mean_absolute_error: 0.2160 - 22ms/epoch - 11ms/step
Epoch 4962/5000
2/2 - 0s - loss: 0.1556 - mean_squared_error: 0.1556 - mean_absolute_error: 0.2122 - 22ms/epoch - 11ms/step
Epoch 4963/5000
2/2 - 0s - loss: 0.1597 - mean_squared_error: 0.1597 - mean_absolute_error: 0.2086 - 21ms/epoch - 10ms/step
Epoch 4964/5000
2/2 - 0s - loss: 0.1669 - mean_squared_error: 0.1669 - mean_absolute_error: 0.2303 - 22ms/epoch - 11ms/step
Epoch 4965/5000
2/2 - 0s - loss: 0.1520 - mean_squared_error: 0.1520 - mean_absolute_error: 0.2033 - 22ms/epoch - 11ms/step
Epoch 4966/5000
2/2 - 0s - loss: 0.1506 - mean_squared_error: 0.1506 - mean_absolute_error: 0.1985 - 22ms/epoch - 11ms/step
Epoch 4967/5000
2/2 - 0s - loss: 0.1537 - mean_squared_error: 0.1537 - mean_absolute_error: 0.2094 - 24ms/epoch - 12ms/step
Epoch 4968/5000
2/2 - 0s - loss: 0.1534 - mean_squared_error: 0.1534 - mean_absolute_error: 0.2065 - 32ms/epoch - 16ms/step
Epoch 49

In [27]:
##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict esults on test data
pred_test = model.predict(X_test)

1/1 [==============================] - 0s 21ms/step


In [28]:
##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")


-------------------- Model Summary --------------------
Model: "LSTM-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 10, 1)             0         
                                                                 
 Input-Layer (InputLayer)    multiple                  0         
                                                                 
 Hidden-LSTM-Encoder-Layer (  (None, 64)               8704      
 Bidirectional)                                                  
                                                                 
 Repeat-Vector-Layer (Repeat  (None, 10, 64)           0         
 Vector)                                                         
                                                                 
 Hidden-LSTM-Decoder-Layer (  (None, 10, 64)           24832     
 Bidirectional)                                                  

In [29]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

#Trace for actual temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns),
                         y=np.array(df_test.values).flatten(),
                         mode='lines',
                         name='Subsidence - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Trace for predicted temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns[-timestep:]),
                         y=pred_test.flatten(),
                         mode='lines',
                         name='Subsidence - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Month-Year'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Subsidence'
                )

# Set figure title
fig.update_layout(title=dict(text="Subsidence", font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )
fig.show()

In [22]:
print(np.array(pred_test).flatten())
print(df_test.columns[-2*timestep:])

[-33.072056 -33.20186  -33.081154 -33.090656 -33.142323 -33.163063
 -33.211452 -33.28171  -33.375587 -33.78608  -33.0761   -33.21275
 -33.082317 -33.095104 -33.14693  -33.167305 -33.21558  -33.285805
 -33.379856 -33.799763 -33.069916 -33.188633 -33.07975  -33.08592
 -33.137722 -33.159077 -33.20781  -33.278095 -33.37067  -33.766525
 -33.06248  -33.179478 -33.079285 -33.082623 -33.134403 -33.155857
 -33.204456 -33.274685 -33.367126 -33.75722  -33.072037 -33.202164
 -33.08153  -33.091362 -33.143196 -33.163914 -33.212296 -33.28254
 -33.375893 -33.786884 -33.077557 -33.198353 -33.08067  -33.089798
 -33.141613 -33.162846 -33.211704 -33.282085 -33.374645 -33.775913
 -33.06947  -33.195217 -33.08082  -33.088863 -33.140755 -33.161724
 -33.21022  -33.280445 -33.373203 -33.77763  -33.073017 -33.1871
 -33.079227 -33.085075 -33.136868 -33.158455 -33.207413 -33.27778
 -33.37009  -33.761086 -33.06699  -33.180283 -33.078724 -33.082333
 -33.134037 -33.155697 -33.204525 -33.274834 -33.367348 -33.754498
 